# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [3]:
! pip install -q schedule pytest bs4 # установка библиотек, если ещё не

In [16]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule #type: ignore
import json
from datetime import datetime
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [ ]:
def get_book_data(book_url: str) -> dict:
    """
    Extracts book information from a Books to Scrape page.
    
    Parses a book page and collects all available data including title,
    price, rating, availability, description, and additional characteristics
    from the Product Information table.
    
    Args:
        url (str): URL of the book page on Books to Scrape website.
    
    Returns:
        dict: Dictionary containing book data with the following keys:
            - title (str): Book title
            - price (str): Book price
            - rating (str): Book rating (One, Two, Three, Four, Five)
            - availability (str): Availability information
            - description (str): Book description (if available)
            - product_info (dict): Additional characteristics from Product
              Information table (UPC, Product Type, Price excl/incl tax,
              Tax, Number of reviews)
    
    Raises:
        requests.RequestException: If an error occurs during page request.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    
    response = requests.get(book_url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    book_data = {}
    
    title_tag = soup.find('h1')
    book_data['title'] = title_tag.text.strip() if title_tag else None
    
    price_tag = soup.find('p', class_='price_color')
    book_data['price'] = price_tag.text.strip() if price_tag else None
    
    rating_tag = soup.find('p', class_='star-rating')
    if rating_tag:
        rating_classes = rating_tag.get('class', [])
        book_data['rating'] = rating_classes[1] if len(rating_classes) > 1 else None
    else:
        book_data['rating'] = None
    
    availability_tag = soup.find('p', class_='instock availability')
    if availability_tag:
        book_data['availability'] = availability_tag.text.strip()
    else:
        book_data['availability'] = None
    
    description_tag = soup.find('div', id='product_description')
    if description_tag:
        description_p = description_tag.find_next_sibling('p')
        book_data['description'] = description_p.text.strip() if description_p else None
    else:
        book_data['description'] = None
    
    product_info = {}
    table = soup.find('table', class_='table table-striped')
    if table:
        rows = table.find_all('tr')
        for row in rows:
            header = row.find('th')
            data = row.find('td')
            if header and data:
                key = header.text.strip()
                value = data.text.strip()
                product_info[key] = value
    
    book_data['product_info'] = product_info
    
    return book_data

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ 

In [8]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'availability': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [14]:
def scrape_books(is_save=False, max_pages=None):
    """
    Scrapes book data from all catalog pages on Books to Scrape website.
    Iterates through all catalog pages and collects information about each book
    using the get_book_data function. Optionally saves results to a file.
    
    Args:
        save_to_file (bool): If True, saves results to 'books_data.txt' file
                            in the same directory. Defaults to False.
        max_pages (int or None): Maximum number of pages to scrape. 
                                 If None, scrapes all available pages.
                                 Defaults to None.
    
    Returns:
        list: List of dictionaries, where each dictionary contains
              data about one book collected by get_book_data function.
    
    Raises:
        requests.RequestException: If an error occurs during page requests.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ

    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    books_list = []
    page_num = 1
    
    print("Starting to scrape books...")
    
    while True:
        if max_pages and page_num > max_pages:
            print(f"Reached maximum pages limit: {max_pages}")
            break
        
        catalog_url = base_url.format(page_num)
        print(f"Processing page {page_num}: {catalog_url}")
        
        try:
            response = requests.get(catalog_url)
            if response.status_code == 404:
                print(f"Page {page_num} not found. Finished scraping.")
                break
            
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            book_articles = soup.find_all('article', class_='product_pod')
            
            if not book_articles:
                print(f"No books found on page {page_num}. Stopping.")
                break
            
            for article in book_articles:
                h3_tag = article.find('h3')
                if h3_tag:
                    a_tag = h3_tag.find('a')
                    if a_tag and a_tag.get('href'):
                        book_relative_url = a_tag['href']
                        book_url = f"http://books.toscrape.com/catalogue/{book_relative_url}"
                        
                        try:
                            book_data = get_book_data(book_url)
                            books_list.append(book_data)
                            print(f"  Scraped: {book_data['title']}")
                        except Exception as e:
                            print(f"  Error scraping {book_url}: {e}")
                
                        time.sleep(0.1)
            
            page_num += 1
            
        except requests.RequestException as e:
            print(f"Error requesting page {page_num}: {e}")
            break
    
    print(f"\nTotal books scraped: {len(books_list)}")
    if is_save:
        with open('books_data.txt', 'w', encoding='utf-8') as f:
            json.dump(books_list, f, ensure_ascii=False, indent=2)

    return books_list

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [15]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Starting to scrape books...
Processing page 1: http://books.toscrape.com/catalogue/page-1.html
  Scraped: A Light in the Attic
  Scraped: Tipping the Velvet
  Scraped: Soumission
  Scraped: Sharp Objects
  Scraped: Sapiens: A Brief History of Humankind
  Scraped: The Requiem Red
  Scraped: The Dirty Little Secrets of Getting Your Dream Job
  Scraped: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
  Scraped: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
  Scraped: The Black Maria
  Scraped: Starving Hearts (Triangular Trade Trilogy, #1)
  Scraped: Shakespeare's Sonnets
  Scraped: Set Me Free
  Scraped: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
  Scraped: Rip it Up and Start Again
  Scraped: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
  Scraped: Olio
  Scraped: Mesaerion: The Best Science Fiction Stories 1800-1849
  Scraped: Libertarianism for Beg

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [27]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

def run_scheduler(schedule_time="19:00"):
    """
    Sets up and runs the scheduler for automatic book scraping.
    
    Configures daily scraping at the specified time and runs an infinite
    loop that checks for pending tasks every 60 seconds to avoid system
    overload. When the scheduled time is reached, performs scraping and
    saves data to file.
    
    Args:
        schedule_time (str): Time in HH:MM format when scraping should run.
                            Defaults to "19:00".
    
    Returns:
        None: This function runs indefinitely until manually stopped.
    """
    def scraping_job():
        """Internal function that performs the actual scraping."""
        print(f"\n{'='*60}")
        print(f"Scheduled scraping started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*60}\n")
        
        try:
            books = scrape_books(is_save=True)
            print(f"\n{'='*60}")
            print(f"Scraping completed successfully at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Total books collected: {len(books)}")
            print(f"{'='*60}\n")
        except Exception as e:
            print(f"\n{'='*60}")
            print(f"Error during scheduled scraping: {e}")
            print(f"{'='*60}\n")
    
    schedule.every().day.at(schedule_time).do(scraping_job)
    
    print(f"Scheduler initialized. Books will be scraped daily at {schedule_time}")
    print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("Press Ctrl+C to stop the scheduler\n")
    
    try:
        while True:
            schedule.run_pending()
            time.sleep(60)
    except KeyboardInterrupt:
        print("\nScheduler stopped by user")

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [19]:
test_time = '19:00'
print(f"TEST MODE: Scheduling scraping for {test_time}")
run_scheduler(schedule_time=test_time)

TEST MODE: Scheduling scraping for 19:00
Scheduler initialized. Books will be scraped daily at 19:00
Current time: 2025-11-09 21:19:34
Press Ctrl+C to stop the scheduler


Scheduler stopped by user


In [28]:
test_time2 = '21:30'
print(f"TEST MODE: Scheduling scraping for {test_time2}")
run_scheduler(schedule_time=test_time2)

TEST MODE: Scheduling scraping for 21:30
Scheduler initialized. Books will be scraped daily at 21:30
Current time: 2025-11-09 21:29:11
Press Ctrl+C to stop the scheduler


Scheduled scraping started at 2025-11-09 21:30:11

Starting to scrape books...
Processing page 1: http://books.toscrape.com/catalogue/page-1.html
  Scraped: A Light in the Attic
  Scraped: Tipping the Velvet
  Scraped: Soumission
  Scraped: Sharp Objects
  Scraped: Sapiens: A Brief History of Humankind
  Scraped: The Requiem Red
  Scraped: The Dirty Little Secrets of Getting Your Dream Job
  Scraped: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
  Scraped: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
  Scraped: The Black Maria

Scheduler stopped by user


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [5]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

]9;4;3;\============================= test session starts ==============================
platform darwin -- Python 3.12.10, pytest-9.0.0, pluggy-1.5.0
rootdir: /Users/eloevv/repositories/MIPT_HW_Books_Scraper
collected 17 items                                                             

tests/test_scraper.py ]9;4;1;0\.]9;4;1;5\.]9;4;1;11\.]9;4;1;17\.]9;4;1;23\.]9;4;1;29\.]9;4;1;35\.]9;4;1;41\.]9;4;1;47\.]9;4;1;52\.]9;4;1;58\.]9;4;1;64\.]9;4;1;70\.]9;4;1;76\.]9;4;1;82\.]9;4;1;88\.]9;4;1;94\.                                  [100%]]9;4;0;\

======================== 17 passed in 124.24s (0:02:04) ========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```